# Django ORM CookBook

## 정보를 조회하고 필요한 항목을 선별하는 방법

### 1. 장고 ORM이 실행하는 실제 SQL 질의문을 확인할 수 있나요?

In [1]:
from events.models import Event

In [2]:
queryset = Event.objects.all()
str(queryset.query)

'SELECT "events_event"."id", "events_event"."epic_id", "events_event"."details", "events_event"."years_ago" FROM "events_event"'

In [3]:
queryset = Event.objects.filter(years_ago__gt=5)
str(queryset.query)

'SELECT "events_event"."id", "events_event"."epic_id", "events_event"."details", "events_event"."years_ago" FROM "events_event" WHERE "events_event"."years_ago" > 5'

### 2. OR 연산으로 일부 조건을 하나라도 만족하는 항목을 구하려면 어떻게 하나요?

In [21]:
# 데이터 베이스 생성
from django.contrib.auth import get_user_model
get_user_model()

data = [
    ['yash', 'Yash', 'Rastogi', 'yashr@agiliq.com'],
    ['John', 'John', 'Kumar', 'john@gmail.com'],
    ['Ricky', 'Ricky', 'Dayal', 'ricky@gmail.com'],
    ['sharukh', 'Sharukh', 'Misra', 'sharukh@hotmail.com'],
    ['Ritesh', 'Ritesh', 'Deshmukh', 'ritesh@yahoo.com'],
    ['Billy', 'Billy', 'Sharma', 'billy@gmail.com'],
    ['Radha', 'Radha', 'George', 'radha@gmail.com'],
    ['sohan' ,'Sohan', 'Upadhyay', 'sohan@aol.com'],
    ['Raghu', 'Raghu', 'Khan', 'raghu@rediffmail.com'],
    ['rishab', 'Rishabh', 'Deol', 'rishabh@yahoo.com']
]

from django.contrib.auth.models import User

for i in data:
    User.objects.create(
        username=i[0],
        first_name=i[1],
        last_name=i[2],
        email=i[3])

`OR` 연산 // 이름이 'R'로 시작하거나, 성이 'D'로 시작하는 모든 사용자 구하기
- `queryset_1 | queryset_2`
- `filter(Q(<condition_1>)|Q(<condition_2>))`

구하려는 SQL 질의문:  
`SELECT username, first_name, last_name, email FROM auth_user WHERE first_name LIKE 'R%' OR last_name LIKE 'D%';`


In [27]:
'''
- queryset_1 | queryset_2
'''

queryset = User.objects.filter(first_name__startswith='R') | User.objects.filter(last_name__startswith='D')
queryset

<QuerySet [<User: Ricky>, <User: Ritesh>, <User: Radha>, <User: Raghu>, <User: rishab>]>

In [28]:
str(queryset.query)

'SELECT "auth_user"."id", "auth_user"."password", "auth_user"."last_login", "auth_user"."is_superuser", "auth_user"."username", "auth_user"."first_name", "auth_user"."last_name", "auth_user"."email", "auth_user"."is_staff", "auth_user"."is_active", "auth_user"."date_joined" FROM "auth_user" WHERE ("auth_user"."first_name" LIKE R% ESCAPE \'\\\' OR "auth_user"."last_name" LIKE D% ESCAPE \'\\\')'

In [31]:
'''
- filter(Q(<condition_1>)|Q(<condition_2>))
'''

from django.db.models import Q
qs = User.objects.filter(Q(first_name__startswith='R')|Q(last_name__startswith='D'))

In [32]:
str(qs.query)

'SELECT "auth_user"."id", "auth_user"."password", "auth_user"."last_login", "auth_user"."is_superuser", "auth_user"."username", "auth_user"."first_name", "auth_user"."last_name", "auth_user"."email", "auth_user"."is_staff", "auth_user"."is_active", "auth_user"."date_joined" FROM "auth_user" WHERE ("auth_user"."first_name" LIKE R% ESCAPE \'\\\' OR "auth_user"."last_name" LIKE D% ESCAPE \'\\\')'

### 3. AND 연산으로 여러 조건을 모두 만족하는 항목을 구하려면 어떻게 하나요?

`AND` 연산 // 이름이 'R'로 시작하고 성이 'D'로 시작하는 모든 사용자 구하기

- `filter(<condition_1>, <condition_2>)`
- `queryset_1 & queryset_2`
- `filter(Q(<condition_1>) & Q(<condition_2>))`

구하려는 SQL 질의문:  
`SELECT username, first_name, last_name, email FROM auth_user WHERE first_name LIKE 'R%' AND last_name LIKE 'D%';`

In [33]:
'''
filter(<condition_1>, <condition_2>)
'''
qs1 = User.objects.filter(
    first_name__startswith='R',
    last_name__startswith='D'
    )

In [36]:
'''
queryset_1 & queryset_2
'''
qs2 = User.objects.filter(
    first_name__startswith='R'
    ) & User.objects.filter(
    last_name__startswith='D'
    )

In [37]:
'''
filter(Q(<condition_1>) & Q(<condition_2>))
'''
qs3 = User.objects.filter(
    Q(first_name__startswith='R') &
    Q(last_name__startswith='D')
    )

In [42]:
str(qs1) == str(qs2) == str(qs3)

True

In [44]:
str(qs1.query)

'SELECT "auth_user"."id", "auth_user"."password", "auth_user"."last_login", "auth_user"."is_superuser", "auth_user"."username", "auth_user"."first_name", "auth_user"."last_name", "auth_user"."email", "auth_user"."is_staff", "auth_user"."is_active", "auth_user"."date_joined" FROM "auth_user" WHERE ("auth_user"."first_name" LIKE R% ESCAPE \'\\\' AND "auth_user"."last_name" LIKE D% ESCAPE \'\\\')'

### 4. NOT 연산으로 조건을 부정하려면 어떻게 하나요?

id < 5 라는 조건을 만족하지 않는 모든 사용자를 구해 봅시다.  
이를 수행하려면 NOT 연산이 필요합니다.

- `exclude(<condition>)`
- `filter(~Q(<condition>))`

구하려는 SQL 질의문:  
`SELECT id, username, first_name, last_name, email FROM auth_user WHERE NOT id < 5;`

In [50]:
qs1 = User.objects.exclude(id__lt=5)
qs1

<QuerySet [<User: Ritesh>, <User: Billy>, <User: Radha>, <User: sohan>, <User: Raghu>, <User: rishab>]>

In [51]:
qs2 = User.objects.filter(~Q(id__lt=5))
qs2

<QuerySet [<User: Ritesh>, <User: Billy>, <User: Radha>, <User: sohan>, <User: Raghu>, <User: rishab>]>

### 5. 동일한 모델 또는 서로 다른 모델에서 구한 쿼리셋들을 합할 수 있나요?

SQL에서는 여러 개의 결과 집합을 합할 때 UNION 연산을 이용합니다.  
장고 ORM에서 union 메서드를 이용해 쿼리셋을 합할 수 있습니다.  
합하려는 쿼리셋의 모델이 서로 다른 경우, 각 쿼리셋에 포함된 필드와 데이터 유형이 서로 맞아야 합니다.

In [52]:
q1 = User.objects.filter(id__gte=5)
q1

<QuerySet [<User: Ritesh>, <User: Billy>, <User: Radha>, <User: sohan>, <User: Raghu>, <User: rishab>]>

In [53]:
q2 = User.objects.filter(id__lte=9)
q2

<QuerySet [<User: yash>, <User: John>, <User: Ricky>, <User: sharukh>, <User: Ritesh>, <User: Billy>, <User: Radha>, <User: sohan>, <User: Raghu>]>

In [54]:
q1.union(q2)

<QuerySet [<User: yash>, <User: John>, <User: Ricky>, <User: sharukh>, <User: Ritesh>, <User: Billy>, <User: Radha>, <User: sohan>, <User: Raghu>, <User: rishab>]>

In [55]:
q2.union(q1)

<QuerySet [<User: yash>, <User: John>, <User: Ricky>, <User: sharukh>, <User: Ritesh>, <User: Billy>, <User: Radha>, <User: sohan>, <User: Raghu>, <User: rishab>]>

In [56]:
q3 = EventVillain.objects.all()
q3

<QuerySet []>

In [57]:
q1.union(q3)

OperationalError: SELECTs to the left and right of UNION do not have the same number of result columns

쿼리셋의 필드와 데이터 유형이 서로 다르기 때문에 union 실패.  
공통된 필드만 가져온 뒤 union을 수행할 수 있음

In [59]:
Hero.objects.all().values_list(
    "name", "gender"
).union(
Villain.objects.all().values_list(
    "name", "gender"))

<QuerySet []>

### 6. 필요한 열만 골라 조회하려면 어떻게 하나요?

- 쿼리셋의 `values` 메서드와 `values_list` 메서드
- `only` 메서드

이름이 R로 시작하는 모든 사용자의 이름(first_name)과 성(last_name)을 구해 봅시다.  
데이터베이스 시스템의 부하를 줄이기 위해 그 외의 열은 가져오지 않겠습니다.

In [66]:
'''
- values
'''

qs1 = User.objects.filter(
    first_name__startswith='R'
    ).values('first_name', 'last_name')

In [65]:
str(qs1.query)

'SELECT "auth_user"."first_name", "auth_user"."last_name" FROM "auth_user" WHERE "auth_user"."first_name" LIKE R% ESCAPE \'\\\''

In [68]:
qs2 = User.objects.filter(
    first_name__startswith='R'
    ).only('first_name', 'last_name')
str(qs2.query)

'SELECT "auth_user"."id", "auth_user"."first_name", "auth_user"."last_name" FROM "auth_user" WHERE "auth_user"."first_name" LIKE R% ESCAPE \'\\\''

### 7. 장고에서 서브쿼리 식을 사용할 수 있나요?

장고에서 SQL 서브쿼리(subquery, 질의문 내의 하위 질의) 식을 사용할 수 있습니다.

In [70]:
'''
auth_user 모델과 일 대 일(OneToOne) 관계로 연결된 UserParent 모델이 있다고 합시다.
UserParent 모델에서 auth_user 를 가진 행을 모두 구할 수 있습니다.
'''

from django.db.models import Subquery

users = User.objects.all()
UserParent.objects.filter(user_id__in=Subquery(users.values('id')))

<QuerySet []>

Category 모델의 각 행 별로, 가장 선한 Hero 행을 구하기.

In [81]:
'''
benevolence_factor에 따라 내림차순으로 정렬
category=OuterRef('pk')으로 서브쿼리로 사용할 준비

OuterRef: 서브쿼리의 queryset이 외부 쿼리의 필드를 참조할 경우 사용
참조: https://docs.djangoproject.com/en/3.0/ref/models/expressions/
'''
hero_qs = Hero.objects.filter(
    category=OuterRef('pk')
    ).order_by('-benevolence_factor')

'''
most_benevolent_hero=Subquery()
서브쿼리에 별칭을 붙여 Category 쿼리셋 안에서 사용

hero_qs.values('name')[:1]
서브쿼리에서 첫 번째 행의 name 필드를 구함

annotate: queryset의 각 오브젝트에 쿼리 표현식 주석을 닮
>>> q = Blog.objects.annotate(number_of_entries=Count('entry'))
>>> q[0].number_of_entries
42
참조: https://docs.djangoproject.com/en/3.0/ref/models/querysets/#django.db.models.query.QuerySet.annotate
'''
qs = Category.objects.all().annotate(
      most_benevolent_hero=Subquery(
         hero_qs.values('name')[:1]
        )
    )

In [82]:
'''
SELECT "entities_category"."id",
       "entities_category"."name",

  (SELECT U0."name"
   FROM "entities_hero" U0
   WHERE U0."category_id" = ("entities_category"."id")
   ORDER BY U0."benevolence_factor" DESC
   LIMIT 1) AS "most_benevolent_hero"
FROM "entities_category"

'''
str(qs.query)

'SELECT "entities_category"."id", "entities_category"."name", (SELECT U0."name" FROM "entities_hero" U0 WHERE U0."category_id" = ("entities_category"."id") ORDER BY U0."benevolence_factor" DESC  LIMIT 1) AS "most_benevolent_hero" FROM "entities_category"'

### 8. 필드의 값을 서로 비교하여 항목을 선택할 수 있나요?

이름(first_name) 이 'R' 로 시작하는 User 모델의 행을 구하려면   `User.objects.filter(first_name__startswith='R')`와 같이 코드를 작성
  
필드와 필드를 서로 비교
  
예를 들어, 이름(first_name) 을 성(last_name) 과 비교하여 선택하는 것이죠.  
이럴 때 F 객체를 사용합니다.

In [83]:
# 실습을 위한 샘플 데이터 생성
User.objects.create_user(email="shabda@example.com", username="shabda", first_name="Shabda", last_name="Raaj")
User.objects.create_user(email="guido@example.com", username="Guido", first_name="Guido", last_name="Guido")

<User: Guido>

In [84]:
# 이름과 성이 동일한 사용자
User.objects.filter(last_name=F('first_name'))

<QuerySet [<User: Guido>]>

`F` 객체는 annotate 메서드로 계산해 둔 필드를 가리킬 때도 사용 가능  
예를 들어, 이름의 첫 글자와 성의 첫 글자가 동일한 사용자를 구하고 싶다면 `Substr("first_name", 1, 1)` 를 사용할 수 있음

In [85]:
User.objects.create_user(email="guido@example.com", username="Tim", first_name="Tim", last_name="Teters")

<User: Tim>

In [89]:
'''
class Substr(expression, pos, lengh=None, **extra)
field나 expression에서 pos부터 length까지의 문자열을 리턴
참조: https://docs.djangoproject.com/en/3.0/ref/models/database-functions/
'''
from django.db.models.functions import Substr

User.objects.annotate(first=Substr('first_name', 1, 1), last=Substr('last_name', 1, 1)).filter(first=F('last'))

<QuerySet [<User: Guido>, <User: Tim>]>

`F` 갹체에 `__gt`, `__lt` 등의 룩업 적용 가능